# LSTM Experiment

In [ ]:
import os
import re

from loguru import logger
import numpy as np
import pandas as pd
import seaborn as sns
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader

from utils import FeatUtils

%load_ext autoreload
%autoreload 2

sns.set_theme(style="dark")

# 1. Prepare features

In [ ]:
prefix = "./data/"
group = "train"
X, y = FeatUtils.load_dataset_group(group, prefix)
# Decrease label's value by one to match the index of prediction outputs
y["label"] = y["label"] - 1

# Show class stat
n_row = len(y)
for i in np.unique(y):
    n_label = len(y.loc[y["label"] == i])
    print(f"Class {i}: {n_label} rows {(n_label / n_row) * 100}%")

In [ ]:
split_frac = 0.8
X_train, X_valid, X_test, y_train, y_valid, y_test = FeatUtils.make_train_valid_test_feature(
    X, y, prep_func=None, split_frac=split_frac)
## print out the shapes of your resultant feature data
print("Train set:", X_train.shape)
print("Validation set:", X_valid.shape)
print("Test set:", X_test.shape)

In [ ]:
# Create DataLoader
batch_size = 16
train_loader, valid_loader, test_loader = FeatUtils.make_dataloaders(X_train, X_valid, X_test, y_train, y_valid, y_test, batch_size=batch_size)

dataiter = iter(train_loader)
sample_x, sample_y = dataiter.next()
print("Sample input size: ", sample_x.size()) # batch_size, seq_length, input_size
print()
print("Sample label size: ", sample_y.size()) # batch_size

# 2. Define Network structure

In [ ]:
from models import HarLSTM, ModelUtils

# check if GPU is available
train_on_gpu = torch.cuda.is_available()
if(train_on_gpu):
    print('Training on GPU!')
else: 
    print('No GPU available, training on CPU; consider making n_epochs very small.')

In [ ]:
# Instantiate the model w/ hyperparams
input_size = 9
output_size = len(np.unique(y))
n_hidden = 128
n_layers = 2

# training params
epochs = 50
lr=0.0001

net = HarLSTM(input_size, output_size, n_hidden=n_hidden, n_layers=n_layers)

print("Model information:")
print(net)

# loss and optimization functions
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=lr)

# 3. Train the model

In [ ]:
train_stat_dict = ModelUtils.train_net(net, criterion, optimizer, train_loader, valid_loader, batch_size, epochs, 
                            train_on_gpu=train_on_gpu, print_every=100, clip=5)

In [ ]:
ModelUtils.plot_loss_chart(train_stat_dict)

In [ ]:
# Save model weights
model_path = f"har_lstm_{batch_size}_ep{epochs}.pt"
ModelUtils.save_model_weight(net, model_path)

# 4. Test inference by a loaded model

In [ ]:
loaded_net = HarLSTM(input_size, output_size, n_hidden=n_hidden, n_layers=n_layers)
ModelUtils.load_model_weight(loaded_net, model_path)

In [ ]:
test_loss, test_acc = ModelUtils.test_net(loaded_net, criterion, test_loader, batch_size, train_on_gpu=train_on_gpu)